In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import numpy as np

In [2]:
from label import label

In [3]:
df = label('../binance-BTCUSDT-1m.csv', t=1)

In [4]:
df

,Open,High,Low,Close,Volumn,Timestamp End,Quote_asset_volume,Number_of_trades,Base_asset_volume,Quote_asset_volume,Time_UTC_Start,label
Timestamp,,,,,,,,,,,,
1531910040000,7394.60,7394.60,7382.07,7385.00,27.539346,1531910099999,2.034712e+05,217,10.883414,8.040716e+04,2018-07-18 10:34:00,0.0
1531910100000,7385.00,7385.00,7378.27,7379.18,16.894401,1531910159999,1.247068e+05,184,10.320014,7.618136e+04,2018-07-18 10:35:00,0.0
1531910160000,7378.27,7379.18,7370.00,7372.47,44.158140,1531910219999,3.256312e+05,229,21.079610,1.554834e+05,2018-07-18 10:36:00,0.0
1531910220000,7370.03,7377.00,7367.22,7373.95,46.129979,1531910279999,3.399929e+05,300,33.766038,2.488708e+05,2018-07-18 10:37:00,1.0
1531910280000,7373.95,7374.00,7369.33,7370.00,11.282937,1531910339999,8.318349e+04,142,7.941090,5.855096e+04,2018-07-18 10:38:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1651909740000,35920.00,35923.50,35913.03,35913.04,12.096760,1651909799999,4.345057e+05,388,7.151330,2.568724e+05,2022-05-07 07:49:00,0.0
1651909800000,35913.04,35922.47,35913.03,35922.46,24.311140,1651909859999,8.731911e+05,575,9.917070,3.561851e+05,2022-05-07 07:50:00,1.0
1651909860000,35922.46,35950.19,35922.46,35940.01,72.302490,1651909919999,2.598789e+06,782,32.884100,1.181822e+06,2022-05-07 07:51:00,1.0


In [73]:
import gc
def base_feat(df, base_cols=[], 
                  time_delta=1, 
                  window_lookback=[1, 7, 24, 99, 500],
                  agg_func=['min', 'max', 'mean', 'median', 'var']):
    """
    This func use for create formal features base on a list of columns
    Params:
        df (Pandas DF): dataframe input
        base_cols (list): list of columns you want to create feature by this function
        time_delta (int): time between 2 records (minutes)
        window_lookback (list): list of integer time the time_delta. For example time_delta = 15 window_lookback = [1, 2]
                                so the time lookback is 15 minutes and 15*2=30 minutes
        agg_func (list): aggregate func you use for rolling
    """
    
    for lookback in window_lookback:
        for col in base_cols:
            dict_agg_func = {'_'.join([agg, col, str(time_delta * lookback) + 'M']): agg for agg in agg_func}
            df_feat = df[col].rolling(lookback).agg(dict_agg_func)
            df = pd.concat([df, df_feat], axis=1)
            del df_feat
            gc.collect()
    for col in df.columns:
        if '1' in col and 'var' in col:
            del df[col]
    df = df.dropna()
    return df

In [74]:
base_feat(df, base_cols=['Volumn'])

,Open,High,Low,Close,Volumn,Timestamp End,Quote_asset_volume,Number_of_trades,Base_asset_volume,Quote_asset_volume,...,min_Volumn_99M,max_Volumn_99M,mean_Volumn_99M,median_Volumn_99M,var_Volumn_99M,min_Volumn_500M,max_Volumn_500M,mean_Volumn_500M,median_Volumn_500M,var_Volumn_500M
Timestamp,,,,,,,,,,,,,,,,,,,,,
1531939980000,7420.34,7431.45,7417.03,7423.22,49.272967,1531940039999,3.657977e+05,323,32.497162,2.412778e+05,...,6.727896,391.215625,67.537782,42.641448,4572.560924,3.749802,391.215625,42.801821,29.710498,1903.984309
1531940040000,7423.26,7430.00,7420.55,7420.56,22.526784,1531940099999,1.672242e+05,176,12.512042,9.288298e+04,...,6.727896,391.215625,67.677329,42.641448,4557.903194,3.749802,391.215625,42.791795,29.710498,1904.341190
1531940100000,7420.56,7432.46,7420.55,7432.43,40.919488,1531940159999,3.039202e+05,244,17.975724,1.335268e+05,...,6.727896,391.215625,67.982680,42.641448,4531.976353,3.749802,391.215625,42.839846,30.061657,1903.001864
1531940160000,7432.41,7446.18,7425.75,7429.38,51.819428,1531940219999,3.852357e+05,261,20.818641,1.548004e+05,...,6.727896,391.215625,68.335188,44.938104,4507.911723,3.749802,391.215625,42.855168,30.061657,1903.159735
1531940220000,7429.38,7447.91,7426.00,7432.05,36.123147,1531940279999,2.685722e+05,159,28.286022,2.103456e+05,...,6.727896,391.215625,68.500361,44.938104,4494.405998,3.749802,391.215625,42.835154,30.061657,1903.228663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651909740000,35920.00,35923.50,35913.03,35913.04,12.096760,1651909799999,4.345057e+05,388,7.151330,2.568724e+05,...,1.036860,325.715380,19.564851,11.588310,1216.776466,1.036860,325.715380,15.236969,10.275990,410.051687
1651909800000,35913.04,35922.47,35913.03,35922.46,24.311140,1651909859999,8.731911e+05,575,9.917070,3.561851e+05,...,1.036860,325.715380,19.704108,11.697870,1216.152812,1.036860,325.715380,15.270309,10.333190,410.099958
1651909860000,35922.46,35950.19,35922.46,35940.01,72.302490,1651909919999,2.598789e+06,782,32.884100,1.181822e+06,...,1.036860,325.715380,20.400667,11.931480,1241.161757,1.036860,325.715380,15.393637,10.333190,416.560106
